In [67]:
import pandas as pd

In [68]:
df = pd.read_csv('battle_results.csv')

df['p1win'] = df['result'] == 1
df['p2win'] = df['result'] == 2
df['tie'] = df['result'] == 0

df = df.drop(columns=['Unnamed: 0'])

df.head(5)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
0,0,0,2,scyther,scyther,False,True,False
1,0,1,2,scyther,pinsir,False,True,False
2,0,2,2,scyther,moltres,False,True,False
3,0,3,2,scyther,moltres EX,False,True,False
4,0,4,2,scyther,heatmor,False,True,False


In [69]:
evo_names = df.groupby('player1')['p1name'].first().reset_index().rename(columns={'player1': 'id', 'p1name': 'name'})
evo_names.head(5)

,id,name
0,0,scyther
1,1,pinsir
2,2,moltres
3,3,moltres EX
4,4,heatmor


In [70]:
def get_losses(id:int) -> set[int]:
    p1 = df[(df['player1'] == id) & ~(df['p1win'])]
    p2 = df[(df['player2'] == id) & ~(df['p2win'])]
    return set([int(item) for item in set(p1['player2'].unique()) | set(p2['player1'].unique())])

In [71]:
loss74 = get_losses(74)
loss75 = get_losses(75)

eggLosses = loss74 & loss75

named_evo_list = lambda s: [evo_names.loc[item, 'name'].title() for item in s]

print(f"Common losses:           {named_evo_list(eggLosses)}")
print(f"Egg 74 unique losses:    {named_evo_list(loss74 - eggLosses)}")
print(f"Egg 75 unique losses:    {named_evo_list(loss75 - eggLosses)}")

Common losses:           ['Seaking', 'Exeggutor Ex', 'Rampardos', 'Mismagius Ex', 'Infernape Ex']
Egg 74 unique losses:    ['Starmie Ex', 'Glaceon', 'Exeggutor Ex', 'Glaceon', 'Mismagius Ex', 'Rapidash', 'Rapidash', 'Salazzle']
Egg 75 unique losses:    ['Moltres Ex', 'Primape', 'Dialga Ex']


In [72]:
max_battles = int((df['player1'].max() + 1) * 2) - 1
max_battles

615

In [73]:
(df['result'].value_counts() / df['result'].count() * 100).round(2)

result
2    60.20
1    39.65
0     0.15
Name: count, dtype: float64

In [74]:
player_wins = df.groupby('player1')[['p1win', 'tie']].sum().rename(columns={'p1win': 'player_win', 'tie': "player_tie"})
opponent_wins = df[df['player1'] != df['player2']].groupby('player2')[['p2win', 'tie']].sum().rename(columns={'p2win': 'opponent_win', 'tie': "opponent_tie"})

wins = player_wins.merge(opponent_wins, left_index=True, right_index=True).reset_index().rename(columns={'player1': 'id'})
wins['wins'] = wins['player_win'] + wins['opponent_win']

wins = wins.set_index('id').merge(evo_names.set_index('id'), right_index=True, left_index=True).reset_index()

wins['win_perc'] = (wins['wins'] / max_battles * 100).round(2)

wins.head(5)

,id,player_win,player_tie,opponent_win,opponent_tie,wins,name,win_perc
0,0,49,0,94,0,143,scyther,23.25
1,1,128,0,209,0,337,pinsir,54.80
2,2,116,0,228,0,344,moltres,55.93
3,3,227,0,278,0,505,moltres EX,82.11
4,4,64,0,102,0,166,heatmor,26.99


In [75]:
wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False)

,id,wins,win_perc,name
75,75,608,98.86,exeggutor EX
74,74,602,97.89,exeggutor EX
300,300,594,96.59,Rampardos
209,209,583,94.80,mismagius EX
63,63,578,93.98,dialga EX
...,...,...,...,...
48,48,0,0.00,manaphy
39,39,0,0.00,chatot
52,52,0,0.00,mespirit
147,147,0,0.00,aerodactyl


In [76]:
wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False).head(20)

,id,wins,win_perc,name
75,75,608,98.86,exeggutor EX
74,74,602,97.89,exeggutor EX
300,300,594,96.59,Rampardos
209,209,583,94.80,mismagius EX
63,63,578,93.98,dialga EX
210,210,578,93.98,mismagius EX
217,217,577,93.82,weavile EX
99,99,576,93.66,starmie EX
72,72,572,93.01,exeggutor
286,286,570,92.68,Infernape Ex


In [77]:
list(set(named_evo_list(get_losses(75))))

['Moltres Ex',
 'Dialga Ex',
 'Infernape Ex',
 'Seaking',
 'Rampardos',
 'Exeggutor Ex',
 'Mismagius Ex',
 'Primape']

In [78]:
df[((df['player1'] == 75) & ~(df['p1win'])) | ((df['player2'] == 75) & ~(df['p2win']))]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
23103,75,3,2,exeggutor EX,moltres EX,False,True,False
23163,75,63,2,exeggutor EX,dialga EX,False,True,False
23175,75,75,1,exeggutor EX,exeggutor EX,True,False,False
23226,75,126,2,exeggutor EX,primape,False,True,False
23310,75,210,2,exeggutor EX,mismagius EX,False,True,False
29951,97,75,1,seaking,exeggutor EX,True,False,False
88163,286,75,1,Infernape Ex,exeggutor EX,True,False,False
92475,300,75,1,Rampardos,exeggutor EX,True,False,False


In [79]:
wins[['id', 'player_win', 'name']].sort_values('player_win', axis=0, ascending=False).head(20)

,id,player_win,name
300,300,305,Rampardos
75,75,304,exeggutor EX
74,74,301,exeggutor EX
217,217,293,weavile EX
294,294,288,Togekiss
73,73,285,exeggutor
72,72,284,exeggutor
286,286,284,Infernape Ex
261,261,284,Kabutops
234,234,284,Beedrill


In [80]:
df[df['tie']]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
3099,10,19,0,pikachu EX,hitmonlee,False,False,True
3196,10,116,0,pikachu EX,zebstrika,False,False,True
5862,19,10,0,hitmonlee,pikachu EX,False,False,True
5871,19,19,0,hitmonlee,hitmonlee,False,False,True
5878,19,26,0,hitmonlee,ditto,False,False,True
...,...,...,...,...,...,...,...,...
51294,166,166,0,lumineon,lumineon,False,False,True
69674,226,66,0,lickilicky EX,fan rotom,False,False,True
80421,261,33,0,Kabutops,mew EX,False,False,True
80649,261,261,0,Kabutops,Kabutops,False,False,True
